# 🤗 Quanto: a pytorch quantization toolkit

Welcome to this tutorial where we showcase how to use `quanto` library quantize any model in 8, 4, even 2 bit precision on GPU / CPU and MPS device! Let's get started 🔥

### Setup

In [ ]:
pip install quanto accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 10.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from quanto import freeze, quantize
import torch

In [ ]:
# monkey patched for quanto
def named_module_tensors(module, recurse=False):
    for named_parameter in module.named_parameters(recurse=recurse):
      name, val = named_parameter
      flag = True
      if hasattr(val,"_data") or hasattr(val,"_scale"):
        if hasattr(val,"_data"):
          yield name + "._data", val._data
        if hasattr(val,"_scale"):
          yield name + "._scale", val._scale
      else:
        yield named_parameter

    for named_buffer in module.named_buffers(recurse=recurse):
      yield named_buffer

def dtype_byte_size(dtype):
    """
    Returns the size (in bytes) occupied by one parameter of type `dtype`.
    """
    import re
    if dtype == torch.bool:
        return 1 / 8
    bit_search = re.search(r"[^\d](\d+)$", str(dtype))
    if bit_search is None:
        raise ValueError(f"`dtype` is not a valid dtype: {dtype}.")
    bit_size = int(bit_search.groups()[0])
    return bit_size // 8

def compute_module_sizes(model):
    """
    Compute the size of each submodule of a given model.
    """
    from collections import defaultdict
    module_sizes = defaultdict(int)
    for name, tensor in named_module_tensors(model, recurse=True):
      size = tensor.numel() * dtype_byte_size(tensor.dtype)
      name_parts = name.split(".")
      for idx in range(len(name_parts) + 1):
        module_sizes[".".join(name_parts[:idx])] += size

    return module_sizes

### Without quantization

Let's first load following model `bigscience/bloom-560m` and play with it.

In [ ]:
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m", low_cpu_mem_usage=True, torch_dtype = torch.float32).to("cuda")
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

In [ ]:
print(model)

In [ ]:
text = "Hello my name is"

inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

You can see that the model size is 2.2GB (~560m parameters * 4 bytes) since we loaded the model in torch.float32 (32 bits = 4 bytes).

In [ ]:
module_sizes = compute_module_sizes(model.transformer)
print(module_sizes)
print(f"The model size is {module_sizes[''] * 1e-9} GB")

In [ ]:
print(model.transformer.h[0].self_attention.query_key_value.weight)

### 8-bit quantization (weight only)

Now, we will use quanto to quantize the model in 8 bits. To do that, we can specify that we want the weights to be quantized with the quanto data type `qint8`. In this example, we will only quantize the weights but we can also quantize the activation.

#### 1. Quantize
At this stage, only the inference of the model is modified to dynamically quantize the weights. (aka dynamic quantization)

In [ ]:
from quanto import qint8
# quantization happens in-place
quantize(model.transformer, weights=qint8, activations=None)

You can see that the `Linear` layers have been replaced by `QLinear` from quanto library. Also, we don't quantize the `lm_head`, so that we don't get performance degradation with LLMs.

In [ ]:
print(model)

In [ ]:
print(model.transformer.h[0].self_attention.query_key_value.weight)

In [ ]:
module_sizes = compute_module_sizes(model.transformer)

print(f"The model size is {module_sizes[''] * 1e-9} GB")

As you can see, the weights are still on torch.float32 and we quantize the weights only during inference.
This intermediate state is useful for:
- **Calibration** : Record the activation ranges while passing representative samples through the quantized model
- **Quantization-Aware-Training** : If the performance of the model degrades too much, one can tune it for a few epochs to recover the float model performance


We can also dynamically quantize the weights by calling the property `qweight`. This is what is done in practice during inference if we don't freeze the weights.

In [ ]:
print(model.transformer.h[0].self_attention.query_key_value.qweight)

#### 2. Freeze the weights (float -> int)

When freezing a model, its float weights are replaced by quantized integer weights.

In [ ]:
freeze(model)

You can see that we have integer weights now

In [ ]:
print(model.transformer.h[0].self_attention.query_key_value.weight)

The model size is only 1.3 GB now ! Note that the embedding layer takes 1GB and is not quantized. This is why the model size is not divided by 4 as expected if the entire model is converted to from 32 bits to 8 bits. You can see that by doing: `print(module_sizes)`

In [ ]:
module_sizes = compute_module_sizes(model.transformer)
print(f"The model size is {module_sizes[''] * 1e-9} GB")

In [ ]:
text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=10)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))